
# Diagnose von Diabetes mellitus mithilfe maschinellen Lernens

Hier finden Sie den gesamten Code, den der Autor für die Analyse verwendet hat.

Auch einzusehen unter: https://github.com/NicoPiel/LiteraturSeminar

In [1]:
# Setup

import numpy as np

# Ein Zufalls-Seed für Reproduzierbarkeit
np.random.seed(42)

from datetime import datetime
import pandas as pd
import seaborn as sns
from scipy import stats
import sklearn as sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, classification_report
from sklearn.impute import KNNImputer
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks, utils

from IPython.display import display, clear_output

sns.set_theme()

print(f"GPUs für die Berechnung: {len(tf.config.experimental.list_physical_devices('GPU'))}")

/home/nico/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nico/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nico/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nico/anaconda3/envs/tensorflow/lib

GPUs für die Berechnung: 0


/home/nico/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nico/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nico/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nico/anaconda

In [2]:
# Daten importieren
data = pd.read_csv('data/diabetes.csv')

# Nullen durch NaN ersetzen
data["Glucose"] = data["Glucose"].replace(0, np.nan)
data["BloodPressure"] = data["BloodPressure"].replace(0, np.nan)
data["SkinThickness"] = data["SkinThickness"].replace(0, np.nan)
data["Insulin"] = data["Insulin"].replace(0, np.nan)
data["BMI"] = data["BMI"].replace(0, np.nan)
data["DiabetesPedigreeFunction"] = data["DiabetesPedigreeFunction"].replace(0, np.nan)
data["Age"] = data["Age"].replace(0, np.nan)

# NaN Werte imputieren
imputer = KNNImputer(n_neighbors=3)

data_imputed = pd.DataFrame(imputer.fit_transform(data))
data_imputed.columns = data.columns

# Vorbereitung

Als Erstes möchten wir uns ein wenig Übersicht verschaffen, also Data Exploration betreiben.

In [3]:
# Spalten

data_imputed.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [4]:
# Übersicht verschaffen

data_imputed.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6.0,148.0,72.0,35.0,126.0,33.6,0.627,50.0,1.0
1,1.0,85.0,66.0,29.0,106.0,26.6,0.351,31.0,0.0
2,8.0,183.0,64.0,33.0,325.0,23.3,0.672,32.0,1.0
3,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21.0,0.0
4,0.0,137.0,40.0,35.0,168.0,43.1,2.288,33.0,1.0


In [5]:
data_imputed.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,2768.000000,2768.000000,2768.000000,2768.000000,2768.000000,2768.000000,2768.000000,2768.000000,2768.000000
mean,3.742775,121.865968,72.386802,29.394870,154.245303,32.571038,0.471193,33.132225,0.343931
std,3.323801,30.691555,12.405154,10.878381,113.830777,7.143838,0.325669,11.777230,0.475104
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,64.000000,22.000000,76.000000,27.500000,0.244000,24.000000,0.000000
50%,3.000000,117.000000,72.000000,30.000000,125.000000,32.300000,0.375000,29.000000,0.000000
75%,6.000000,141.000000,80.000000,36.000000,190.000000,36.725000,0.624000,40.000000,1.000000
max,17.000000,199.000000,122.000000,110.000000,846.000000,80.600000,2.420000,81.000000,1.000000


Die einzelnen Spalten haben folgende Bedeutung:

- **Pregnancies:** Anzahl der bisherigen (erfolgreichen und gescheiterten) Schwangerschaften
- **Glucose:** 75g-2-h-oGTT in $\frac{\text{mg}}{\text{dl}}$. Glukosewert nach 75g Kohlenhydraten und zwei Stunden Wartezeit
- **BloodPressure:** Diastolischer Blutdruck in $\text{mm} \cdot \text{Hg}$
- **SkinThickness:** Dicke der Trizepshautfalte in $\text{mm}$
- **Insulin:** Seruminsulin nach 2 Stunden in $\frac{\text{µU}}{\text{ml}}$
- **BMI:** Body-Mass-Index in $\frac{\text{kg}}{\text{m}^2}$
- **DiabetesPedigreeFunction:** Stammbaumfunktion des Patienten
- **Age:** Alter in Jahren
- **Outcome:** Klassifikation. 1 = Diabetes, 0 = kein Diabetes

In [6]:
# Keine sichtbare Beziehung zwischen

#sns.pairplot(data = data_imputed, hue="Outcome")
#plt.savefig('pairplot.png', dpi=300)

In [7]:
# Pearson-Koeffizienten für alle Spalten

for col in data_imputed:
    for col2 in data_imputed:
        if col != col2:
            arr1 = data_imputed[col].to_numpy()
            arr2 = data_imputed[col2].to_numpy()
        
            pearson = stats.pearsonr(arr1, arr2)
            
            if (pearson[0] > 0.3) or (pearson[0] < -0.3):
                print(f"{col} -> {col2}: r = {np.round(pearson[0], 2)}, p = {np.round(pearson[1], 2)}")

Pregnancies -> Age: r = 0.54, p = 0.0
Glucose -> Insulin: r = 0.51, p = 0.0
Glucose -> Outcome: r = 0.49, p = 0.0
BloodPressure -> Age: r = 0.33, p = 0.0
SkinThickness -> BMI: r = 0.54, p = 0.0
Insulin -> Glucose: r = 0.51, p = 0.0
BMI -> SkinThickness: r = 0.54, p = 0.0
Age -> Pregnancies: r = 0.54, p = 0.0
Age -> BloodPressure: r = 0.33, p = 0.0
Outcome -> Glucose: r = 0.49, p = 0.0


Aus den Pearson-Koeffizienten lässt sich schließen, dass die klinischen Werte nur wenig direkten Einfluss aufeinander haben.

Die Anzahl der Schwangerschaften scheint mit dem Alter, die Dicke der Trizeps-Hautfalte mit dem BMI zu korrelieren.
Überraschenderweise scheint der 75g-2h-oGGT nur mittelmäßig mit dem Ergebnis zusammenzuhängen.

# Das Modell

In [8]:
def nn(random_state):
    # Input-Daten
    X = data_imputed[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']].values
    # Der Wert, der vorhergesagt werden soll
    y = data_imputed['Outcome'].values

    # Train-Test-Split als 60/40
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)

    # Das Modell soll aufhören zu rechnen, falls es keine nennenswerten Verbesserungen mehr gibt
    early_stopping = callbacks.EarlyStopping(
        min_delta=0.001,
        patience=64,
        restore_best_weights=True
    )

    # Das NN besteht aus einer Mischung von Dense-, Normalization- und Dropout-Layern.
    network = keras.Sequential([
        layers.Dense(8, activation='relu', input_shape=[X_train.shape[1]]),
        layers.Dense(8, activation='relu'),
        layers.Dense(8, activation='relu'),
        layers.Dense(8, activation='relu'),
        layers.Dense(8, activation='relu'),
        layers.Dense(8, activation='relu'),
        layers.Dense(8, activation='relu'),
        layers.Dense(1, activation='sigmoid', name='output'),
    ])

    # NN kompilieren
    network.compile(
        optimizer='sgd',
        loss='binary_crossentropy',
        metrics=[keras.metrics.BinaryAccuracy()]
    )

    history = network.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        batch_size=128,
        epochs=512,
        shuffle=True,
        callbacks=[early_stopping],
        verbose=0
    )

    # Scores berechnen
    # history_df = pd.DataFrame(history.history)

    y_pred = network.predict(X_test)
    y_pred_corrected = []

    for pred in y_pred:
        if pred > 0.5:
            y_pred_corrected.append(1)
        else: 
            y_pred_corrected.append(0)

    f1 = f1_score(y_test, y_pred_corrected, average='macro')
    report = sklearn.metrics.classification_report(y_test, y_pred_corrected)
    # confusion_matrix = sklearn.metrics.confusion_matrix(y_test, y_pred_corrected)
    # print(confusion_matrix)
    # print(report)
    return f1, report, network

    # history_df.loc[5:, ['binary_accuracy', 'val_binary_accuracy']].plot()
    # history_df.loc[5:, ['loss', 'val_loss']].plot()

In [9]:
# 100 verschiedene Modelle trainieren und in einen DataFrame speichern
# Das kann ein paar Stunden dauern

results = {}

for i in np.arange(5):
    f1, report, model = nn(i)
    results[i] = [model, f1, report]
    clear_output()
    df_results = pd.DataFrame.from_dict(results, orient='index', columns=['model', 'f1', 'report'])
    display(df_results)

,model,f1,report
0,<tensorflow.python.keras.engine.sequential.Seq...,0.738616,precision recall f1-score ...
1,<tensorflow.python.keras.engine.sequential.Seq...,0.716600,precision recall f1-score ...
2,<tensorflow.python.keras.engine.sequential.Seq...,0.764555,precision recall f1-score ...
3,<tensorflow.python.keras.engine.sequential.Seq...,0.714232,precision recall f1-score ...
4,<tensorflow.python.keras.engine.sequential.Seq...,0.572025,precision recall f1-score ...


In [10]:
# DataFrame nach bestem F1-Score sortieren
df_results_sorted = df_results.sort_values(by=['f1'], ascending=False)

# Die besten 10 Modelle speichern
for i in np.arange(5):
    df_results_sorted.iloc[i].model.save(f'models/model{i}.h5')

In [11]:
df_results.describe()

,f1
count,5.000000
mean,0.701206
std,0.075011
min,0.572025
25%,0.714232
50%,0.716600
75%,0.738616
max,0.764555
